In [1]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
# Importing the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import xgboost as xgb

In [3]:
df_orig  = pd.read_csv('Crop Classification 2.csv', encoding = 'utf-8')

In [4]:
#Remove Crop categories with less than 5 % frequency
dataframe_5per = df_orig[df_orig['Crop Name'] != 'Potato']
dataframe_5per = dataframe_5per[dataframe_5per['Crop Name'] != 'Tobacco']
dataframe_5per = dataframe_5per[dataframe_5per['Crop Name'] != 'Sugarcane']
dataframe_5per = dataframe_5per[dataframe_5per['Crop Name'] != 'Fennel']

print('Crop record frequency')
print(dataframe_5per['Crop Name'].value_counts(normalize=True))

Crop record frequency
Wheat       0.313736
Cumin       0.255218
Jowar       0.129166
Castor      0.085018
Maize       0.082013
Rapeseed    0.076822
Gram        0.058026
Name: Crop Name, dtype: float64


In [5]:
# Remove Crop categories with less than 7 % frequency

print('Crop record frequency :')
print(df_orig['Crop Name'].value_counts(normalize=True))

dataframe_7per = dataframe_5per[dataframe_5per['Crop Name'] != 'Gram']

Crop record frequency :
Wheat        0.304874
Cumin        0.248009
Jowar        0.125518
Castor       0.082617
Maize        0.079696
Rapeseed     0.074652
Gram         0.056387
Fennel       0.019911
Sugarcane    0.005044
Tobacco      0.002336
Potato       0.000956
Name: Crop Name, dtype: float64


In [6]:
X = dataframe_7per.iloc[:, 2:35].values
y = dataframe_7per.iloc[:, 1].values
print(X.shape)
print(y.shape)

(17240, 33)
(17240,)


In [7]:
#SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')
X, y = smote.fit_sample(X, y)
print(X.shape)
print(y.shape)

(21576, 33)
(21576,)


In [8]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify = y, random_state = 0)

In [9]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
n_words = X_test.shape[1]

def baseline_model():
    # define network
    model = Sequential()
    model.add(Dense(30, input_shape=(n_words,), activation='relu'))
    model.add(Dense(6, activation='sigmoid'))

    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    estimator = KerasClassifier(build_fn = baseline_model, epochs=200, batch_size=10, verbose=0)
    return model

In [11]:
estimator = KerasClassifier(build_fn = baseline_model, epochs=200, batch_size=5, verbose=0)

In [12]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=7)

In [13]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_test)
dummy_y = np_utils.to_categorical(encoded_Y)

results = cross_val_score(estimator, X_test, dummy_y, cv = kfold)
print(results.mean()*100)

W0727 16:43:11.188840 12424 deprecation_wrapper.py:119] From C:\Users\Kunjan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 16:43:11.254049 12424 deprecation_wrapper.py:119] From C:\Users\Kunjan\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 16:43:11.292826 12424 deprecation_wrapper.py:119] From C:\Users\Kunjan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 16:43:11.293865 12424 deprecation_wrapper.py:119] From C:\Users\Kunjan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 16:43:11.326463 12424 deprecation_wrapper.py:119] From C:\Users\Kunjan

90.42727123000624


In [ ]:
#Case 1 : 3 layer network
#model.add(Dense(50, input_shape=(n_words,), activation='relu'))
#model.add(Dense(50, input_shape=(n_words,), activation='relu'))
#model.add(Dense(50, input_shape=(n_words,), activation='relu'))
#model.add(Dense(6, activation='softmax'))
#Accuracy : 67.93288777984263

#Case 2 : 2 Layer network
#model = Sequential()
#model.add(Dense(50, input_shape=(n_words,), activation='relu'))
#model.add(Dense(50, input_shape=(n_words,), activation='relu'))
#model.add(Dense(6, activation='softmax'))
#Accuracy : 68.9060230729627

#Case 3 : Single Layer network with 50 Neurons
#model.add(Dense(50, input_shape=(n_words,), activation='relu'))
#Accuracy : 71.8494362682229

#Case 4 : Single Layer network with 75 Neurons
#model.add(Dense(75, input_shape=(n_words,), activation='relu'))
#Accuracy : 70.68928975592958